Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [91]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [92]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# FIX: Use the official public API path from pandas.errors
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

# Chapter 8 - Multi-Factor Models

## 8.1 Estimating the CAPM

### How to do it...

1. Import the libraries:

In [97]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

2. Specify the risky asset, the benchmark, and the time horizon:

In [99]:
RISKY_ASSET = "AMZN"
MARKET_BENCHMARK = "^GSPC"
START_DATE = "2016-01-01"
END_DATE = "2020-12-31"

3. Download data from Yahoo Finance:

In [101]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 end=END_DATE,
                 auto_adjust=True,  # FIX: Replaced 'adjusted' with 'auto_adjust'
                 progress=False)

print(f'Downloaded {df.shape[0]} rows of data.')

Downloaded 1258 rows of data.


4. Resample to monthly data and calculate simple returns:

In [103]:
X = (
    df["Close"]
    .rename(columns={RISKY_ASSET: "asset", 
                     MARKET_BENCHMARK: "market"})
    .resample("M")
    .last()
    .pct_change()
    .dropna()
)
X.head()

Ticker,asset,market
Date,,
2016-02-29,-0.058739,-0.004128
2016-03-31,0.074423,0.065991
2016-04-30,0.111094,0.002699
2016-05-31,0.095817,0.015325
2016-06-30,-0.009920,0.000911


5. Calculate beta using the covariance approach: 

In [105]:
covariance = X.cov().iloc[0,1]
benchmark_variance = X.market.var()
beta = covariance / benchmark_variance
beta

np.float64(1.153710787431552)

6. Prepare the input and estimate CAPM as a linear regression:

In [107]:
# separate target
y = X.pop("asset")

# add constant
X = sm.add_constant(X)

# define and fit the regression model 
capm_model = sm.OLS(y, X).fit()

# print results 
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     36.15
Date:                Sun, 20 Jul 2025   Prob (F-statistic):           1.37e-07
Time:                        17:05:05   Log-Likelihood:                 79.803
No. Observations:                  59   AIC:                            -155.6
Df Residuals:                      57   BIC:                            -151.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0188      0.009      2.182      0.0

Or, using the formula notation:

In [109]:
import yfinance as yf
import statsmodels.formula.api as smf

# Step 1: Define assets and download the data into the 'df' DataFrame
RISKY_ASSET = "AMZN"
MARKET_BENCHMARK = "^GSPC"
START_DATE = "2016-01-01"
END_DATE = "2020-12-31"

df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 end=END_DATE,
                 auto_adjust=True,
                 progress=False)

# Step 2: Now process the 'df' DataFrame
# FIX: Select the 'Close' column, which contains the adjusted prices
X = (
    df["Close"]
    .rename(columns={RISKY_ASSET: "asset", 
                     MARKET_BENCHMARK: "market"})
    .resample("ME")
    .last()
    .pct_change()
    .dropna()
)

# define and fit the regression model 
capm_model = smf.ols(formula="asset ~ market", data=X).fit()

# print results 
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     36.15
Date:                Sun, 20 Jul 2025   Prob (F-statistic):           1.37e-07
Time:                        17:05:05   Log-Likelihood:                 79.803
No. Observations:                  59   AIC:                            -155.6
Df Residuals:                      57   BIC:                            -151.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0188      0.009      2.182      0.0

### There's more

#### Risk-free rate (13 Week Treasury Bill)

In [112]:
# period length in days
N_DAYS = 90

# download data from Yahoo finance
df_rf = yf.download("^IRX",
                    start=START_DATE,
                    end=END_DATE,
                    progress=False)

# resample to monthly by taking last value from each month
rf = df_rf.resample("M").last().Close / 100

# calculate the corresponding daily risk-free return  
rf = ( 1 / (1 - rf * N_DAYS / 360) )**(1 / N_DAYS)  

# convert to monthly and subtract 1
rf = (rf ** 30) - 1 

# plot the risk-free rate
rf.plot(title="Risk-free rate (13 Week Treasury Bill)")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_2", dpi=200)

#### Risk-free rate (3-Month Treasury Bill)

In [114]:
import pandas_datareader.data as web

# download the data 
rf = web.DataReader(
    "TB3MS", "fred", start=START_DATE, end=END_DATE
)

# convert to monthly
rf = (1 + (rf / 100)) ** (1 / 12) - 1

# plot the risk-free rate
rf.plot(title="Risk-free rate (3-Month Treasury Bill)")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_3", dpi=200)

## 8.2 Estimating the Fama-French three-factor model

### How to do it...

1. Import the libraries:

In [118]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web

2. Define parameters:

In [120]:
RISKY_ASSET = "AAPL"
START_DATE = "2016-01-01"
END_DATE = "2020-12-31"

3. Download the dataset containing the risk factors:

In [122]:
ff_dict = web.DataReader("F-F_Research_Data_Factors", 
                         "famafrench", 
                         start=START_DATE,
                         end=END_DATE)

In [123]:
ff_dict.keys()

dict_keys([0, 1, 'DESCR'])

In [124]:
print(ff_dict['DESCR'])

F-F Research Data Factors
-------------------------

This file was created using the 202505 CRSP database. The 1-month TBill rate data until 202405 are from Ibbotson Associates. Starting from 202406, the 1-month TBill rate is from ICE BofA US 1-Month Treasury Bill Index. Copyright 2025 Eugene F. Fama and Kenneth R. French

  0 : (60 rows x 4 cols)
  1 : Annual Factors: January-December (5 rows x 4 cols)


4. Select the appropriate dataset and divide the values by 100:

In [126]:
factor_3_df = ff_dict[0].rename(columns={"Mkt-RF": "MKT"}) \
                        .div(100)

factor_3_df.head()

,MKT,SMB,HML,RF
Date,,,,
2016-01,-0.0574,-0.0340,0.0208,0.0001
2016-02,-0.0006,0.0073,-0.0061,0.0002
2016-03,0.0695,0.0075,0.0122,0.0002
2016-04,0.0092,0.0077,0.0322,0.0001
2016-05,0.0178,-0.0018,-0.0162,0.0001


5. Download the prices of the risky asset:

In [170]:
asset_df = yf.download(RISKY_ASSET,
                       start=START_DATE,
                       end=END_DATE,
                       auto_adjust=True,  # FIX: Replaced 'adjusted' with 'auto_adjust'
                       progress=False)

print(f"Downloaded {asset_df.shape[0]} rows of data.")

Downloaded 1258 rows of data.


6. Calculate monthly returns on the risky asset:

In [ ]:
y = asset_df["Adj Close"].resample("M") \
                         .last() \
                         .pct_change() \
                         .dropna()

y.index = y.index.to_period("m")
y.name = "rtn"
y.head()

7. Merge the datasets and calculate excess returns:

In [ ]:
factor_3_df = factor_3_df.join(y)
factor_3_df["excess_rtn"] = (
    factor_3_df["rtn"] - factor_3_df["RF"]
)
factor_3_df.head()

8. Estimate the three-factor model:

In [ ]:
# define and fit the regression model 
ff_model = smf.ols(formula="excess_rtn ~ MKT + SMB + HML", 
                   data=factor_3_df).fit()

# print results 
print(ff_model.summary())

### There's more

Print available datasets (here only first 5):

In [ ]:
from pandas_datareader.famafrench import get_available_datasets

get_available_datasets()[:5]

### Bonus

1. Download data from prof. French's website:

To do so, we used the fact that we can execute bash commands in Jupyter Notebooks by preceding them with `!`. First, we downloaded the file using wget and then unzipped it using unzip. There are also ways to do this in Python only, but this seemed like a good place to introduce the possibility of mixing up bash script into the Notebooks. The link to the monthly data is always the same, and the file is updated every month. 


In [ ]:
# download the zip file from Prof. French's website
!wget http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip

# unpack the zip file
!unzip -a F-F_Research_Data_Factors_CSV.zip

# remove the zip file
!rm F-F_Research_Data_Factors_CSV.zip

2. Load data from the source CSV file and keep only the monthly data:

In [ ]:
# load data from CSV
factor_3_df = pd.read_csv("F-F_Research_Data_Factors.csv", skiprows=3)

# identify where the annual data starts
STR_TO_MATCH = " Annual Factors: January-December "
indices = factor_3_df.iloc[:, 0] == STR_TO_MATCH
start_of_annual = factor_3_df[indices].index[0]

# keep only monthly data
factor_3_df = factor_3_df[factor_3_df.index < start_of_annual]

3. Rename columns of the DataFrame, set a datetime index and filter by dates:

In [ ]:
# rename columns
factor_3_df.columns = ["date", "mkt", "smb", "hml", "rf"]

# convert strings to datetime
factor_3_df["date"] = (
    pd.to_datetime(factor_3_df["date"], format="%Y%m")
    .dt.strftime("%Y-%m")
)

# set index
factor_3_df = factor_3_df.set_index("date")

# filter only required dates
factor_3_df = factor_3_df.loc[START_DATE:END_DATE]

4. Convert the values to numeric and divide by 100:

In [ ]:
factor_3_df = factor_3_df.apply(pd.to_numeric, 
                            errors="coerce") \
                     .div(100)
factor_3_df.head()

## 8.3 Estimating the rolling three-factor model on a portfolio of assets

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web

2. Define the parameters:

In [ ]:
ASSETS = ["AMZN", "GOOG", "AAPL", "MSFT"]
WEIGHTS = [0.25, 0.25, 0.25, 0.25]
START_DATE = "2010-01-01"
END_DATE = "2020-12-31"

3. Download the factor related data:

In [ ]:
factor_3_df = web.DataReader("F-F_Research_Data_Factors", 
                             "famafrench", 
                             start=START_DATE,
                             end=END_DATE)[0]
factor_3_df = factor_3_df.div(100)

4. Download the prices of risky assets from Yahoo Finance:

In [ ]:
asset_df = yf.download(ASSETS,
                       start=START_DATE,
                       end=END_DATE,
                       adjusted=True,
                       progress=False)

print(f"Downloaded {asset_df.shape[0]} rows of data.")

5. Calculate the monthly returns on the risky assets:

In [ ]:
asset_df = asset_df["Adj Close"].resample("M") \
                                .last() \
                                .pct_change() \
                                .dropna()
# reformat index for joining
asset_df.index = asset_df.index.to_period("m")

6. Calculate the portfolio returns:

In [ ]:
asset_df["portfolio_returns"] = np.matmul(
    asset_df[ASSETS].values, 
    WEIGHTS
)
asset_df.head()

In [ ]:
asset_df.plot();

7. Merge the datasets:

In [ ]:
factor_3_df = asset_df.join(factor_3_df).drop(ASSETS, axis=1)
factor_3_df.columns = ["portf_rtn", "mkt", "smb", "hml", "rf"]
factor_3_df["portf_ex_rtn"] = (
    factor_3_df["portf_rtn"] - factor_3_df["rf"]
)

8. Define a function for the rolling n-factor model

In [ ]:
def rolling_factor_model(input_data, formula, window_size):
    """
    Function for estimating the Fama-French (n-factor) model using a rolling window of fixed size.
    
    Parameters
    ------------
    input_data : pd.DataFrame
        A DataFrame containing the factors and asset/portfolio returns
    formula : str
        `statsmodels` compatible formula representing the OLS regression  
    window_size : int
        Rolling window length.
    
    Returns
    -----------
    coeffs_df : pd.DataFrame
        DataFrame containing the intercept and the three factors for each iteration.
    """

    coeffs = []

    for start_ind in range(len(input_data) - window_size + 1):        
        end_ind = start_ind + window_size

        # define and fit the regression model 
        ff_model = smf.ols(
            formula=formula, 
            data=input_data[start_ind:end_ind]
        ).fit()
   
        # store coefficients
        coeffs.append(ff_model.params)
    
    coeffs_df = pd.DataFrame(
        coeffs, 
        index=input_data.index[window_size - 1:]
    )

    return coeffs_df

9. Estimate the rolling three-factor model and plot the results:

In [ ]:
MODEL_FORMULA = "portf_ex_rtn ~ mkt + smb + hml"
results_df = rolling_factor_model(factor_3_df, 
                                  MODEL_FORMULA, 
                                  window_size=60)
(
    results_df
    .plot(title = "Rolling Fama-French Three-Factor model",
          style=["-", "--", "-.", ":"])
    .legend(loc="center left",bbox_to_anchor=(1.0, 0.5))
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_6", dpi=200)

## 8.4 Estimating the four- and five-factor models

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web

2. Specify the risky asset and the time horizon:

In [ ]:
RISKY_ASSET = "AMZN"
START_DATE = "2016-01-01"
END_DATE = "2020-12-31"

3. Download the risk factors from prof. French's website:

In [ ]:
# three factors 
factor_3_df = web.DataReader("F-F_Research_Data_Factors", 
                             "famafrench", 
                             start=START_DATE,
                             end=END_DATE)[0]

# momentum factor
momentum_df = web.DataReader("F-F_Momentum_Factor", 
                             "famafrench", 
                             start=START_DATE,
                             end=END_DATE)[0]
      
# five factors
factor_5_df = web.DataReader("F-F_Research_Data_5_Factors_2x3", 
                             "famafrench", 
                             start=START_DATE,
                             end=END_DATE)[0]

4. Download the data of the risky asset from Yahoo Finance:

In [ ]:
asset_df = yf.download(RISKY_ASSET,
                       start=START_DATE,
                       end=END_DATE,
                       adjusted=True,
                       progress=False)

print(f"Downloaded {asset_df.shape[0]} rows of data.")

5. Calculate monthly returns:

In [ ]:
y = asset_df["Adj Close"].resample("M") \
                         .last() \
                         .pct_change() \
                         .dropna()

y.index = y.index.to_period("m")
y.name = "rtn"

6. Merge the datasets for the four-factor models:

In [ ]:
# join all datasets on the index
factor_4_df = factor_3_df.join(momentum_df).join(y)

# rename columns
factor_4_df.columns = ["mkt", "smb", "hml", "rf", "mom", "rtn"]

# divide everything (except returns) by 100
factor_4_df.loc[:, factor_4_df.columns != "rtn"] /= 100

# calculate excess returns
factor_4_df["excess_rtn"] = (
    factor_4_df["rtn"] - factor_4_df["rf"]
)

factor_4_df.head()

7. Merge the datasets for the five-factor models:

In [ ]:
# join all datasets on the index
factor_5_df = factor_5_df.join(y)

# rename columns
factor_5_df.columns = [
    "mkt", "smb", "hml", "rmw", "cma", "rf", "rtn"
]

# divide everything (except returns) by 100
factor_5_df.loc[:, factor_5_df.columns != "rtn"] /= 100

# calculate excess returns
factor_5_df["excess_rtn"] = (
    factor_5_df["rtn"] - factor_5_df["rf"]
)

factor_5_df.head()

8. Estimate the four-factor model:

In [ ]:
four_factor_model = smf.ols(
    formula="excess_rtn ~ mkt + smb + hml + mom", 
    data=factor_4_df
).fit()

print(four_factor_model.summary())

9. Estimate the five-factor model:

In [ ]:
five_factor_model = smf.ols(
    formula="excess_rtn ~ mkt + smb + hml + rmw + cma", 
    data=factor_5_df
).fit()

print(five_factor_model.summary())

## 8.5 Estimating cross-sectional factor models using the Fama-MacBeth regression

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import pandas_datareader.data as web
from linearmodels.asset_pricing import LinearFactorModel

2. Specify the time horizon:

In [ ]:
START_DATE = "2010"
END_DATE = "2020-12"

3. Download and adjust the risk factors from prof. French's website:

In [ ]:
factor_5_df = (
    web.DataReader("F-F_Research_Data_5_Factors_2x3", 
                   "famafrench", 
                   start=START_DATE, 
                   end=END_DATE)[0]
    .div(100)
)
factor_5_df.head()

4. Download and adjust the returns of 12 Industry Portfolios from prof. French's website:

In [ ]:
portfolio_df = (
    web.DataReader("12_Industry_Portfolios", 
                   "famafrench", 
                   start=START_DATE, 
                   end=END_DATE)[0]
    .div(100)
    .sub(factor_5_df["RF"], axis=0)
)
portfolio_df.head()

5. Drop the risk-free rate from the factor data set:

In [ ]:
factor_5_df = factor_5_df.drop("RF", axis=1)
factor_5_df.head()

6. Estimate the Fama-MacBeth regression and print the summary:

In [ ]:
five_factor_model = LinearFactorModel(
    portfolios=portfolio_df, 
    factors=factor_5_df
)
result = five_factor_model.fit()
print(result)

We can also print the full summary (1 aggregate and 12 individual ones for each portfolio separately).

In [ ]:
print(result.full_summary)

### There's more

1. Import the libraries:

In [ ]:
from statsmodels.api import OLS, add_constant

2. First step - estimate the factor loadings:

In [ ]:
factor_loadings = []
for portfolio in portfolio_df:
    reg_1 = OLS(
        endog=portfolio_df.loc[:, portfolio], 
        exog=add_constant(factor_5_df)
    ).fit()
    factor_loadings.append(reg_1.params.drop("const"))

3. Store the factor loadings in a DataFrame:

In [ ]:
factor_load_df = pd.DataFrame(
    factor_loadings, 
    columns=factor_5_df.columns, 
    index=portfolio_df.columns
)
factor_load_df.head()

4. Second step - estimate the risk premia:

In [ ]:
risk_premia = []
for period in portfolio_df.index:
    reg_2 = OLS(
        endog=portfolio_df.loc[period, factor_load_df.index], 
        exog=factor_load_df
    ).fit()
    risk_premia.append(reg_2.params)

5. Store the risk premia in a DataFrame:

In [ ]:
risk_premia_df = pd.DataFrame(
    risk_premia, 
    index=portfolio_df.index,
    columns=factor_load_df.columns.tolist())
risk_premia_df.head()

6. Calculate the average risk premia:

In [ ]:
risk_premia_df.mean()